## Co chceme řešit?
Úlohu 2D lineární elasticity.

## Jaké vstupy potřebujeme?
Definovat geometrii, materiálové parametry, okrajové podmínky.

1. Pro definovanou geometrii potřebujeme vytvořit výpočtovou síť. Síť reprezentujeme dvěma dvourozměrnými poli: polem souřadnic uzlů `X` a polem konektivity elementů `IEN`. Tyto pole můžeme pro jednoduché oblasti vytvořit ručně, ale pro složitější geometrie je užitečné použít tzv. síťovač, např. [Gmsh](https://gmsh.info).

```
Vytvoř vstupní soubor pro Gmsh `nosnik.geo`, pro 2D obdélník o délce L = 1 m a šířce b = sqrt(A) m, kde průřez A = 0.1 m^2. Počátek souřadnic je v levém horním rohu a souřadnice b míří ve směru +x a L ve směru -y. Obdélník vysíťuj třemi lineárními quad elementy ve směru y.
```

In [ ]:
warning('off', 'Octave:deprecated-option'); graphics_toolkit('gnuplot'); format compact;

In [ ]:
A = [1 2; 3 4]; det(A); x = linspace(0, 2*pi, 100); y = sin(x); plot(x, y); title('Test Plot');